In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainerCallback, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset, load_dataset
from copy import deepcopy

os.environ["WANDB_DISABLED"] = "true"
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

RANDOM_STATE = 42

In [ ]:
id2label = {0: "CORRECT", 1: "BUGGY"}
label2id = {"CORRECT": 0, "BUGGY": 1}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('dipudl/codet5-base')
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# # Trained model
# model = AutoModelForSequenceClassification.from_pretrained(
#     'dipudl/function-args-swap-bug-distilbert-model',
#     num_labels=2,
#     id2label=id2label,
#     label2id=label2id
# )

In [ ]:
!pip install huggingface_hub
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXXXX')"

-----
## Load Dataset
-----

In [ ]:
train_df = pd.read_csv("/kaggle/input/func-args-swap-dataset/func_args_dataset_filtered_train.csv", sep="\t")
test_df = pd.read_csv("/kaggle/input/func-args-swap-dataset/func_args_dataset_filtered_test.csv", sep="\t")

In [ ]:
train_df.sample(10)

In [ ]:
test_df.sample(5)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.dropna(subset=["function_name", "arg1", "arg2"], inplace=True)
train_df.isna().sum()

In [ ]:
test_df.isna().sum()

In [ ]:
test_df.dropna(subset=["function_name", "arg1", "arg2"], inplace=True)
test_df.isna().sum()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df['param1'] = train_df['param1'].fillna(tokenizer.unk_token)
train_df['param2'] = train_df['param2'].fillna(tokenizer.unk_token)
train_df.isna().sum()

In [ ]:
test_df['param1'] = test_df['param1'].fillna(tokenizer.unk_token)
test_df['param2'] = test_df['param2'].fillna(tokenizer.unk_token)
train_df.isna().sum()

In [ ]:
train_df['full_text'] = train_df['function_name'] + tokenizer.sep_token + train_df['arg1'] + tokenizer.sep_token + train_df['arg2'] + tokenizer.sep_token + train_df['arg_type'] + tokenizer.sep_token + train_df['param1'] + tokenizer.sep_token + train_df['param2']
test_df['full_text'] = test_df['function_name'] + tokenizer.sep_token + test_df['arg1'] + tokenizer.sep_token + test_df['arg2'] + tokenizer.sep_token + test_df['arg_type'] + tokenizer.sep_token + test_df['param1'] + tokenizer.sep_token + test_df['param2']

In [ ]:
train_df.drop(['function_name','arg1','arg2','arg_type','param1','param2'], axis=1, inplace=True)
test_df.drop(['function_name','arg1','arg2','arg_type','param1','param2'], axis=1, inplace=True)

In [ ]:
train_df.sample(5)

In [ ]:
test_df.sample(5)

In [ ]:
print(train_df.iloc[10].full_text)
print("-" * 80)
print(tokenizer.tokenize(train_df.iloc[10].full_text))
print("-" * 80)
print(tokenizer(train_df.iloc[10].full_text))

In [ ]:
def tokenize_text(examples):
    return tokenizer(examples["full_text"], truncation=True, max_length=128, padding=True)

In [ ]:
train_df, validation_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_STATE)
train_df.shape, validation_df.shape

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
train_dataset

In [ ]:
validation_dataset = Dataset.from_pandas(validation_df)
validation_dataset

In [ ]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset

-----
## Tokenization
-----

In [ ]:
train_dataset = train_dataset.map(tokenize_text, batched=True, remove_columns=["full_text", "__index_level_0__"])
train_dataset

In [ ]:
validation_dataset = validation_dataset.map(tokenize_text, batched=True, remove_columns=["full_text", "__index_level_0__"])
validation_dataset

In [ ]:
test_dataset = test_dataset.map(tokenize_text, batched=True, remove_columns=["full_text", "__index_level_0__"])
test_dataset

In [ ]:
def softmax(x):
    result = np.zeros_like(x)
    
    for i in range(len(x)):
        result[i] = np.exp(x[i]) / np.sum(np.exp(x[i]), axis=0)
    return result

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

-----
## Model Training
-----

In [ ]:
# ! rm -rd /kaggle/working/codeT5-DistilBERT-function-swap-bug-model

batch_size = 64
logging_steps = len(train_dataset) // batch_size
output_dir = "codeT5-DistilBERT-function-swap-bug-model"

training_args = TrainingArguments(output_dir,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy="epoch",
                                  logging_steps=logging_steps,
                                  save_strategy="epoch",
                                  # save_steps=10000,
                                  # fp16=True,
                                  load_best_model_at_end=True,
                                  metric_for_best_model='eval_loss',
                                  greater_is_better=False,
                                  push_to_hub=False
                                )

In [ ]:
class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer, test_dataset) -> None:
        super().__init__()
        self._trainer = trainer
        self.test_dataset = test_dataset

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            print("Epoch:", state.epoch)
            print(self._trainer.predict(self._trainer.train_dataset, metric_key_prefix="train").metrics)
            print(self._trainer.predict(self._trainer.eval_dataset, metric_key_prefix="eval").metrics)
            print(self._trainer.predict(self.test_dataset, metric_key_prefix="test").metrics)
            
            train = self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            self._trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")
            return control_copy

In [ ]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,      # number of evaluations to wait before stopping
    early_stopping_threshold=0.01,  # threshold for relative improvement in metric
)

-----
## Choosing Best Hyperparameters and Training Final Model
-----

In [ ]:
learning_rates = [0.0002, 0.002, 0.00002, 0.000002]
batch_sizes = [64, 32]

# Loop over the learning rates
for lr in learning_rates:
    for bs in batch_sizes:
        # Update the learning rate in the TrainingArguments
        training_args.learning_rate = lr
        training_args.per_device_train_batch_size = bs
        training_args.per_device_eval_batch_size = bs

        # Create a new Trainer with the updated TrainingArguments
        trainer = Trainer(
            model=model,                         
            args=training_args,                  
            train_dataset=train_dataset,         
            eval_dataset=validation_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[early_stopping]
        )

        # Train the model and evaluate it on the validation set
        trainer.train()
        print(trainer.evaluate())

-----
## Choosing Best Hyperparameters and Training Final Model
-----

In [ ]:
BEST_LEARNING_RATE = 2e-5
BEST_BATCH_SIZE = 64
training_args.learning_rate = BEST_LEARNING_RATE
training_args.push_to_hub = False
training_args.per_device_train_batch_size = BEST_BATCH_SIZE
training_args.per_device_eval_batch_size = BEST_BATCH_SIZE


trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.add_callback(CustomCallback(trainer, test_dataset)) 

In [ ]:
trainer.train()

In [ ]:
# trainer.save("codeT5-DistilBERT-function-swap-bug-model_5ep")

-----
## Model Testing
-----

In [ ]:
prediction = trainer.predict(test_dataset)

In [ ]:
prediction

In [ ]:
labels = prediction.label_ids
labels

In [ ]:
preds = prediction.predictions.argmax(-1)
preds

In [ ]:
f1_score(labels, preds)

In [ ]:
preds_probability = softmax(prediction.predictions)
positive_preds_probability = preds_probability[:, 1]

In [ ]:
import sklearn.metrics as metrics

fpr, tpr, threshold = metrics.roc_curve(labels, positive_preds_probability)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
precision, recall, threshold = metrics.precision_recall_curve(labels, positive_preds_probability)

# plt.title('Precision-Recall Curve')
plt.plot(recall, precision, 'b')
plt.ylabel('Precision')
plt.xlabel('Recall')

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.show()

In [ ]:
test_df

In [ ]:
test_df["predictions"] = preds

In [ ]:
test_df

In [ ]:
positive_probabilities = softmax(prediction.predictions)[:, 1]
positive_probabilities

In [ ]:
test_df["positive_probabilities"] = positive_probabilities
test_df

In [ ]:
test_df.to_csv("function_args_swap_test_dataset_predictions.csv", sep="\t", index=False)